### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [2]:
from diffusion.data_loaders.motion_dataset import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_cartwheel.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.85536, 0.9966429999999997, -0.0070009999999999795, 0.08157, 0.0005729999999999971, 0.042303731260289315, -0.056088768155961526, -0.01172717680484046, -0.014103614145860938, 0.2358842735659614, 0.37124889801787253, -0.6111023347690597, -0.09268300376873025, -0.09541896434572254, 0.585361, 0.1699928747321186, 0.08652758875118252, 0.354108626550405, 0.160215, -0.2285399691330798, -0.39445967594673703, -0.1178224382194308, -0.369571, 0.20448116583595066, -0.12115992907931128, 0.07892319943485762, 0.3736623102073797, -0.010008232584494297, 0.30603690929303384, -0.364281, -0.13425257761871864, -0.004787718949892447, 0.0010873114649849894] 35


(100,
 Batch(trajectories=tensor([[ 0.0000e+00,  0.0000e+00,  8.5536e-01,  ..., -1.3425e-01,
          -4.7877e-03,  1.0873e-03],
         [ 3.6500e-03, -6.2240e-03,  8.5632e-01,  ..., -1.3036e-01,
           1.4385e-02,  1.5935e-02],
         [ 6.4000e-03, -1.2205e-02,  8.5748e-01,  ..., -1.3041e-01,
           3.6052e-02,  3.1853e-02],
         ...,
         [-2.4664e-01,  2.0540e+00,  8.4651e-01,  ...,  5.9186e-02,
          -2.0091e-01,  2.3318e-01],
         [-2.4351e-01,  2.0658e+00,  8.4672e-01,  ...,  7.3301e-02,
          -2.0666e-01,  2.1504e-01],
         [-2.3938e-01,  2.0782e+00,  8.4690e-01,  ...,  8.5008e-02,
          -2.1333e-01,  1.9921e-01]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.5536e-01,  9.9664e-01, -7.0010e-03,
          8.1570e-02,  5.7300e-04,  4.2304e-02, -5.6089e-02, -1.1727e-02,
         -1.4104e-02,  2.3588e-01,  3.7125e-01, -6.1110e-01, -9.2683e-02,
         -9.5419e-02,  5.8536e-01,  1.6999e-01,  8.6528e-02,  3.5411e-01,
          1.6022e-0

### Setup Model
Configure your experiment name and savepaths here, they will all be stored under the logs folder later on

In [3]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-cartwheel"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [4]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()

/home/kenji/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 160
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [5]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 100
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 160
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 100
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/diffusion_config.pkl



### Setup Training

In [6]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/trainer_config.pkl



### Test if model and training loop works

In [7]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 160, 35])
x.shape torch.Size([1, 35, 160])
x.shape torch.Size([1, 32, 160])
1
xfinal.shape torch.Size([1, 32, 80])
x.shape torch.Size([1, 64, 80])
2
xfinal.shape torch.Size([1, 64, 40])
x.shape torch.Size([1, 128, 40])
3
xfinal.shape torch.Size([1, 128, 20])
x.shape torch.Size([1, 256, 20])
4
xfinal.shape torch.Size([1, 256, 20])
xt1.shape torch.Size([1, 256, 20])
xt2.shape torch.Size([1, 256, 20])
xt3.shape torch.Size([1, 256, 20])
pop.shape torch.Size([1, 256, 20])
pop.shape torch.Size([1, 128, 40])
pop.shape torch.Size([1, 64, 80])


(tensor([[[ 0.0944,  0.1104, -0.1531,  ...,  0.1012,  0.2474,  0.4012],
          [-0.0605,  0.5519, -0.4045,  ..., -0.2518,  0.2368, -0.3138],
          [ 0.0369, -0.1111,  0.0324,  ...,  0.1454,  0.3190,  0.3811],
          ...,
          [-0.0291,  0.3816, -0.2992,  ...,  0.2897,  0.4946,  0.1762],
          [ 0.2478, -0.2832,  0.0067,  ...,  0.2888,  0.1798,  0.4154],
          [ 0.3851,  0.4280,  0.2402,  ...,  0.1765,  0.0856,  0.0039]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 160, 35]))

In [8]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [9]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/state_0.pt
0:   0.3086 | a0_loss:   0.0269 | t:   0.4321
100:   0.0675 | a0_loss:   0.0111 | t:   8.1653
200:   0.0279 | a0_loss:   0.0061 | t:   8.1041
300:   0.0137 | a0_loss:   0.0037 | t:   7.9897
400:   0.0073 | a0_loss:   0.0024 | t:   8.1874
500:   0.0044 | a0_loss:   0.0016 | t:   8.3597
600:   0.0030 | a0_loss:   0.0011 | t:   8.4126
700:   0.0022 | a0_loss:   0.0008 | t:   8.2396
800:   0.0017 | a0_loss:   0.0006 | t:   8.4983
900:   0.0014 | a0_loss:   0.0004 | t:   8.5166
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/state_1.pt


### (Alternatively) load a checkpoint

In [9]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(1)
model = trainer.ema_model

### Sample from model
We want to see what the model does if it starts with a motion, could it convert one motion to another? Or blend them in someway?

Start from existing motion data instead of noise, we want to mix walking and doing a cartwheel

In [10]:
from diffusion.data_loaders.motion_dataset import MotionDataset

walk_dataset = MotionDataset("data/motions/humanoid3d_walk.txt")
len(walk_dataset), walk_dataset[0], walk_dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.847532, 0.998678, 0.01410399999999995, 0.049422999999999995, -0.0006980000000000042, 0.019374995056800275, 0.008037254877450587, -0.09523902811084285, -0.0, 0.0, -0.0, -0.15553532463598624, 0.23919429256424163, 0.20739656997070288, 0.170571, 0.3529631848273465, -0.2610682953696931, -0.24560532144975333, 0.581348, 0.02035205257945668, -0.5175742452141794, -0.11376339039728192, -0.249116, 0.020556236630260034, -0.019534498786735962, 0.0655269812790598, -0.05606350142619236, 0.15209578259548684, 0.1827420948157945, -0.391532, 0.1931167851688944, -0.2978918547932108, -0.08305715225197069] 35


(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.1931, -0.2979, -0.0831],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.1812, -0.2662, -0.1258],
         [ 0.0817, -0.0105,  0.8483,  ...,  0.1293, -0.1646, -0.1874],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ...,  0.0799, -0.2361, -0.1045],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0750, -0.2538, -0.1148],
         [ 1.0080,  0.0138,  0.8721,  ...,  0.0762, -0.2713, -0.1198]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02])}

Extend the walk to be the same shape as the cartwheel

In [11]:
walk_traj = walk_dataset[0].trajectories
# padding = torch.zeros((100, 59))
walk_traj = torch.cat([walk_traj] * 5, dim=0)
walk_traj.unsqueeze_(0)
walk_traj.shape

torch.Size([1, 160, 35])

In [16]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions, starting_motion=walk_traj, max_timesteps=1)


                                                                                                    
1 / 1 [############################################################] 100% | 26.8 Hz
t : 0 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
[ Progress ] 1 / 1 | t : 0 | vmax : 0.0 | vmin : 0.0 | 26.8 Hz


### Save results to logs

In [21]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="mix-walk-motion.npy")

Motion saved as mix-walk-motion2.npy
